<a href="https://colab.research.google.com/github/cagBRT/RNNs/blob/main/3_RNNs_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/WillKoehrsen/recurrent-neural-networks/blob/master/notebooks/Quick%20Start%20to%20Recurrent%20Neural%20Networks.ipynb

https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470

https://victorzhou.com/blog/intro-to-rnns/

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML

InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

import pandas as pd
import numpy as np
from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest

ImportError: ignored

In [ ]:
data = pd.read_csv('../data/neural_network_patent_query.csv')
data.head()

In [ ]:
training_dict, word_idx, idx_word, sequences = get_data('../data/neural_network_patent_query.csv', training_len = 50)


In [ ]:
training_dict['X_train'][:2]
training_dict['y_train'][:2]


In [ ]:
for i, sequence in enumerate(training_dict['X_train'][:2]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text) + '\n')
    print('Label: ' + idx_word[np.argmax(training_dict['y_train'][i])] + '\n')

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model


In [ ]:

model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(word_idx) + 1, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.models import load_model

# Load in model and demonstrate training
model = load_model('../models/train-embeddings-rnn.h5')
h = model.fit(training_dict['X_train'], training_dict['y_train'], epochs = 5, batch_size = 2048, 
          validation_data = (training_dict['X_valid'], training_dict['y_valid']), 
          verbose = 1)

In [ ]:
model = load_model('../models/train-embeddings-rnn.h5')
print('Model Performance: Log Loss and Accuracy on training data')
model.evaluate(training_dict['X_train'], training_dict['y_train'], batch_size = 2048)

print('\nModel Performance: Log Loss and Accuracy on validation data')
model.evaluate(training_dict['X_valid'], training_dict['y_valid'], batch_size = 2048)

In [ ]:
for i in generate_output(model, sequences, idx_word, seed_length = 50, new_words = 30, diversity = 0.75):
    HTML(i)


In [ ]:
for i in generate_output(model, sequences, idx_word, seed_length = 30, new_words = 30, diversity = 1.5):
    HTML(i)

In [ ]:
s = 'This patent provides a basis for using a recurrent neural network to '
HTML(seed_sequence(model, s, word_idx, idx_word, diversity = 0.75, num_words = 20))

In [ ]:
s = 'The cell state is passed along from one time step to another allowing the '
HTML(seed_sequence(model, s, word_idx, idx_word, diversity = 0.75, num_words = 20))

In [ ]:
guess_human(model, sequences, idx_word)


In [ ]:
guess_human(model, sequences, idx_word)

In [ ]:
guess_human(model, sequences, idx_word)


In [ ]:

embeddings = get_embeddings(model)
embeddings.shape

In [ ]:
find_closest('network', embeddings, word_idx, idx_word)

In [ ]:
find_closest('data', embeddings, word_idx, idx_word)
